In [1]:
!pip install transformers datasets transformers[torch] accelerate>=0.20.11


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import load_dataset, load_metric
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, pipeline

/home/grinkevich/venv/main/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.empty_cache()

## Создадим кастомный Trainer для дистилляции знаний

---



1. Определим гиперпараметры α and T

α - на сколько больше мы хотим ориентироваться на предсказания модели-учителя, и меньше на модель-студента  
T - как сильно должно быть сглажено распределение вероятностей классов

2. В качестве модели-учителя будем использовать BERT-base.

3. Новая лосс-функция будет совмещать в себе кросс-энтропию и лосс дистилляции

Чтобы добавить наши гиперпараметры достаточно создать класс TrainingArguments и включить их в него как атрибуты

In [3]:
class KnowledgeDistillationTrainingArguments(TrainingArguments):
  def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
    super().__init__(*args, **kwargs)

    self.alpha = alpha
    self.temperature = temperature

# Напишем лосс-функцию для дистилляции знаний
Создадим наследника класса Trainer и перепишем compute_loss()



In [4]:
class KnowledgeDistillationTrainer(Trainer):
  def __init__(self, *args, teacher_model=None, **kwargs):
    super().__init__(*args, **kwargs)
    self.teacher_model = teacher_model

  def compute_loss(self, model, inputs, return_outputs=False):
    #Extract cross-entropy loss and logits from student
    outputs_student = model(**inputs)
    loss_ce = outputs_student.loss
    logits_student = outputs_student.logits
    inputs.pop('token_type_ids')
    # Extract logits from teacher
    outputs_teacher = self.teacher_model(**inputs)
    logits_teacher = outputs_teacher.logits

    #Computing distillation loss by Softening probabilities
    loss_fct = nn.KLDivLoss(reduction="batchmean")
    #The reduction=batchmean argument in nn.KLDivLoss() specifies that we average the losses over the batch dimension.
    loss_kd = self.args.temperature ** 2 * loss_fct(
                F.log_softmax(logits_student / self.args.temperature, dim=-1),
                F.softmax(logits_teacher / self.args.temperature, dim=-1))

    # Return weighted student loss
    loss = self.args.alpha * loss_ce + (1. - self.args.alpha) * loss_kd
    return (loss, outputs_student) if return_outputs else loss


## Выбираем модель-ученика

Как выбрать подходящую модель-ученика?
1. Меньшая модель чем учитель чтобы уменьшить объем занимаемой памяти и увеличить RPS

2. Дистилляция знаний работает лучше, когда модель-учитель и ученик одного типа (BERT и RoBERTa могут иметь разную длинну эмебддингов на выходе, что создает сложности для ученика мимикрировать под учителя)

В качестве примера на роль модели-ученика возьмем DistilBERT.

### Загрузка датасета

Будем использовать CLINC150 датасет




In [5]:
data = load_dataset("imdb").shuffle()

In [6]:
sample = data["train"][0]
print(sample)

{'text': "This is a great British film. A cleverly observed script with many quotable lines, which captures perfectly what magic mushrooms can do to a man over a weekend. As per usual Phil Daniels is excellent along with that most under rated of British actors Geoff Bell. Peter Bowles with a joint hanging out of his mouth is a casting masterstroke and Gary Stretch with his brooding looks brings something strangely atmospheric to the piece. Although it seems to be billed as a biker movie, i think it will find an audience outside of this, purely on the premise that a lot of people have been there done it and got the t-shirt. also A great original soundtrack with a blinding version of Freebird. This really could be a 21st century heir to the famous Ealing comedies. Like the weed in the Welsh fields: it's a grower!", 'label': 1}


Намерения предоставляются в виде идентификаторов, но мы можем легко получить его значение (и наоборот), вызвав функцию int2str:

# Токенизируем датасет

In [7]:
student_checkpoint = "google/mobilebert-uncased"
student_tokenizer = AutoTokenizer.from_pretrained(student_checkpoint)

In [8]:
def tokenize_text(batch):
  return student_tokenizer(batch["text"], truncation=True, max_length=512)

In [9]:
imdb_tokenized = data.map(tokenize_text, batched=True, remove_columns=["text"])

Map:   4%|▍         | 1000/25000 [00:00<00:02, 9331.73 examples/s]

Map: 100%|██████████| 50000/50000 [00:05<00:00, 9469.71 examples/s] 


# Определим метрику которой будем замерять точность

In [10]:
accuracy_score = load_metric("accuracy")

def compute_metrics(pred):
  predictions, labels = pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy_score.compute(predictions=predictions, references=labels)

/tmp/ipykernel_2124244/3260035165.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_score = load_metric("accuracy")


В этой функции прогнозы от головы модели поступают в форме логитов, поэтому мы используем функцию np.argmax(), чтобы найти наиболее достоверный прогноз класса и сравнить его с ground truth меткой.

# Определим аргументы для тренировки

In [11]:
batch_size = 16
finetuned_student_ckpt = "mobilebert-base-uncased-finetuned-imdb-student"

In [12]:
student_training_args = KnowledgeDistillationTrainingArguments(
    output_dir=finetuned_student_ckpt, evaluation_strategy = "epoch",
    num_train_epochs=1, learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size, alpha=1, weight_decay=0.01)

## Давайте инициализируем модель ученика, но перед этим предоставим ей словари с каждым намерением и его идентификатором.

In [13]:
distilbert_ckpt = "lvwerra/distilbert-imdb"
pipe = pipeline("text-classification", model=distilbert_ckpt)

id2label = pipe.model.config.id2label
label2id = pipe.model.config.label2id

In [14]:
num_labels = 2
student_config = (AutoConfig.from_pretrained(student_checkpoint,
                                             num_labels=num_labels,
                                             id2label=id2label,
                                             label2id=label2id))

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def student_init():
  return (AutoModelForSequenceClassification.from_pretrained(student_checkpoint, config=student_config).to(device))

## Загрузим предобученные веса модели-учителя и начнем дообучение модели-ученика

In [16]:
teacher_model = AutoModelForSequenceClassification.from_pretrained(distilbert_ckpt, num_labels=num_labels, ignore_mismatched_sizes=True).to(device)

In [17]:
# Начинаем fine-tuning ученика
distilbert_trainer = KnowledgeDistillationTrainer(model_init=student_init,
        teacher_model=teacher_model, args=student_training_args,
        train_dataset=imdb_tokenized['train'], eval_dataset=imdb_tokenized['test'],
        compute_metrics=compute_metrics, tokenizer=student_tokenizer)

distilbert_trainer.train()

Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a MobileBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/grinkevich/venv/main/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all inp

Epoch,Training Loss,Validation Loss,Accuracy
1,117327.312000,0.190317,0.927120


/home/grinkevich/venv/main/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=782, training_loss=75017.539643334, metrics={'train_runtime': 415.6524, 'train_samples_per_second': 60.146, 'train_steps_per_second': 1.881, 'total_flos': 1567070261660160.0, 'train_loss': 75017.539643334, 'epoch': 1.0})

## Сравним модели учителя и ученика



Сохраним модели учителя и ученика, а затем вычислим размеры моделей в MB.

In [18]:
teacher_model.save_pretrained("teacher_model")
distilbert_trainer.save_model('student_model')

raw_student = AutoModelForSequenceClassification.from_pretrained(student_checkpoint, config=student_config)
raw_student.save_pretrained("raw_student_model")

Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.dense.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some 

In [19]:
def compute_parameters(model_path):
  model = AutoModelForSequenceClassification.from_pretrained(model_path)
  parameters = model.num_parameters()
  return parameters

In [20]:
teacher_model_parameters = compute_parameters(model_path="./teacher_model")
print("Teacher Model: ", teacher_model_parameters)

Teacher Model:  66955010


In [21]:
student_model_parameters = compute_parameters(model_path="./student_model")
print("Student Model: ", student_model_parameters)

Student Model:  24582914


In [22]:
decrease = (teacher_model_parameters - student_model_parameters) / teacher_model_parameters
print(f'Модель студента имеет на {decrease*100:.2f} % меньше параметров, чем модель учителя')

Модель студента имеет на 63.28 % меньше параметров, чем модель учителя


In [23]:
!ls ./student_model -al --block-size=MB

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
total 100MB
drwxrwxr-x 2 grinkevich grinkevich  1MB Oct  8 11:29 .
drwxrwxr-x 6 grinkevich grinkevich  1MB Oct  8 11:29 ..
-rw-rw-r-- 1 grinkevich grinkevich  1MB Oct  8 17:17 config.json
-rw-rw-r-- 1 grinkevich grinkevich 99MB Oct  8 17:17 pytorch_model.bin
-rw-rw-r-- 1 grinkevich grinkevich  1MB Oct  8 17:17 special_tokens_map.json
-rw-rw-r-- 1 grinkevich grinkevich  1MB Oct  8 17:17 tokenizer.json
-rw-rw-r-- 1 grinkevich grinkevich  1MB Oct  8 17:17 tokenizer_config.json
-rw-rw-r-- 1 grinkevich grinkevich  1MB Oct  8 17:17 training_args.bin
-rw-rw-r-- 1 grinkevich grinkevich  1MB Oct  8 17:17 vocab.txt


In [24]:
!ls ./teacher_model -al --block-size=MB

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
total 268MB
drwxrwxr-x 2 grinkevich grinkevich   1MB Oct  8 11:29 .
drwxrwxr-x 6 grinkevich grinkevich   1MB Oct  8 11:29 ..
-rw-rw-r-- 1 grinkevich grinkevich   1MB Oct  8 17:17 config.json
-rw-rw-r-- 1 grinkevich grinkevich 268MB Oct  8 17:17 pytorch_model.bin


Выполним замер средней скорости инференса у обоих моделей на одинаковых входных данных

In [25]:
sample_input = data['train']['text'][101]

print(data['train']['text'][101])
print(data['train']['label'][101])

I saw this little magnum opus for the first time very recently, on one of those dollar DVD's that seem to be everywhere nowadays, and was so moved by it that I cannot contain myself. For those who have never seen this mesmerizingly miserable Mexican import, and wish to view it without being prejudiced by anyone else's jaundiced commentary, there are undoubtedly substantial spoilers in what follows. So if you are one of those reckless individuals, stop reading at once and go and watch it for yourself. If you get drunk enough in advance, you might be fortunate enough to pass out before it's over.<br /><br />Begin with the premise that a man may become a werewolf after being bitten by a yeti. No one in the film ventures an explanation as to how this sort of cross-species implantation could occur, and the rest of the movie is even more hopelessly nonsensical. But pour yourself another glass of wine (or whatever you're drinking), and let us proceed.<br /><br />Paul Naschy (our werewolf) has

In [28]:
tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}
pipe = pipeline("text-classification", model="./teacher_model", tokenizer='lvwerra/distilbert-imdb')

#WARMUP
for _ in range(10):
  _ = pipe(sample_input, **tokenizer_kwargs)

#INFERENCE
start = time.time()
for _ in range(100):
  _ = pipe(sample_input, **tokenizer_kwargs)
total_time_teacher_model = time.time() - start
print("Общее время обработки 100 запросов моделью-учителем:", total_time_teacher_model)

Общее время обработки 100 запросов моделью-учителем: 8.364979028701782


In [29]:
from tqdm import tqdm

# ПОДСЧЕТ МЕТРИК
data_test_X = data['test']['text'][::50]
data_test_y = data['test']['label'][::50]
model_preds = []

for i in tqdm(data_test_X):
    model_preds.append(label2id[pipe(i, **tokenizer_kwargs)[0]['label']])

accuracy_score.compute(predictions=model_preds, references=data_test_y)

100%|██████████| 500/500 [00:22<00:00, 22.60it/s]


{'accuracy': 0.924}

In [31]:
pipe = pipeline("text-classification", model="./student_model", tokenizer="distilbert-base-uncased")
#WARMUP
for _ in range(10):
  _ = pipe(sample_input,  **tokenizer_kwargs)

#INFERENCE
start = time.time()
for _ in range(100):
  _ = pipe(sample_input, **tokenizer_kwargs)
total_time_student_model = time.time()-start

print("Общее время обработки 100 запросов моделью-студентом:", total_time_student_model)

Общее время обработки 100 запросов моделью-студентом: 7.468947172164917


In [32]:
decrease_in_time = (total_time_teacher_model - total_time_student_model) / total_time_teacher_model

print(f'Модель студента классифицирует быстрее на {decrease_in_time*100:.2f} %')

Модель студента классифицирует быстрее на 10.71 %


In [33]:
# ПОДСЧЕТ МЕТРИК
data_test_X = data['test']['text'][::50]
data_test_y = data['test']['label'][::50]
model_preds = []

for i in tqdm(data_test_X):
    model_preds.append(label2id[pipe(i, **tokenizer_kwargs)[0]['label']])

accuracy_score.compute(predictions=model_preds, references=data_test_y)

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [00:21<00:00, 22.94it/s]


{'accuracy': 0.94}

In [34]:
# Raw student-model
pipe = pipeline("text-classification", model="./raw_student_model", tokenizer="distilbert-base-uncased")

# ПОДСЧЕТ МЕТРИК
data_test_X = data['test']['text'][::50]
data_test_y = data['test']['label'][::50]
model_preds = []

for i in tqdm(data_test_X):
    model_preds.append(label2id[pipe(i, **tokenizer_kwargs)[0]['label']])

accuracy_score.compute(predictions=model_preds, references=data_test_y)

100%|██████████| 500/500 [00:22<00:00, 21.92it/s]


{'accuracy': 0.478}

In [35]:
model_preds

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
